In [570]:
import os
import json
import requests
import pandas as pd
from mysql import connector
from dotenv import load_dotenv

In [571]:
coins = ['bitcoin']

In [572]:
load_dotenv()

True

In [573]:
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "ids": ",".join(coins),
    "order": "market_cap_desc",
    "per_page": len(coins),
    "page": 1,
    "sparkline": "false"
}

In [574]:
print(params)

{'vs_currency': 'usd', 'ids': 'bitcoin', 'order': 'market_cap_desc', 'per_page': 1, 'page': 1, 'sparkline': 'false'}


# EXTRACT

In [575]:
response = requests.get(url, params=params)

In [576]:
payload = response.json()

In [577]:
payload

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
  'current_price': 117167,
  'market_cap': 2334064704577,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 2334064704577,
  'total_volume': 41488303467,
  'high_24h': 117888,
  'low_24h': 116774,
  'price_change_24h': -12.402496997834533,
  'price_change_percentage_24h': -0.01058,
  'market_cap_change_24h': 3757827791,
  'market_cap_change_percentage_24h': 0.16126,
  'circulating_supply': 19922890.0,
  'total_supply': 19922890.0,
  'max_supply': 21000000.0,
  'ath': 124128,
  'ath_change_percentage': -5.63277,
  'ath_date': '2025-08-14T00:37:02.582Z',
  'atl': 67.81,
  'atl_change_percentage': 172644.32472,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2025-09-19T03:04:23.081Z'}]

In [578]:
formatted_response = json.dumps(payload, indent=4)
print(formatted_response)

[
    {
        "id": "bitcoin",
        "symbol": "btc",
        "name": "Bitcoin",
        "image": "https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400",
        "current_price": 117167,
        "market_cap": 2334064704577,
        "market_cap_rank": 1,
        "fully_diluted_valuation": 2334064704577,
        "total_volume": 41488303467,
        "high_24h": 117888,
        "low_24h": 116774,
        "price_change_24h": -12.402496997834533,
        "price_change_percentage_24h": -0.01058,
        "market_cap_change_24h": 3757827791,
        "market_cap_change_percentage_24h": 0.16126,
        "circulating_supply": 19922890.0,
        "total_supply": 19922890.0,
        "max_supply": 21000000.0,
        "ath": 124128,
        "ath_change_percentage": -5.63277,
        "ath_date": "2025-08-14T00:37:02.582Z",
        "atl": 67.81,
        "atl_change_percentage": 172644.32472,
        "atl_date": "2013-07-06T00:00:00.000Z",
        "roi": null,
        "last_u

# TRANSFORM

In [579]:
coins_df = pd.DataFrame(payload)

In [580]:
coins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                1 non-null      object 
 1   symbol                            1 non-null      object 
 2   name                              1 non-null      object 
 3   image                             1 non-null      object 
 4   current_price                     1 non-null      int64  
 5   market_cap                        1 non-null      int64  
 6   market_cap_rank                   1 non-null      int64  
 7   fully_diluted_valuation           1 non-null      int64  
 8   total_volume                      1 non-null      int64  
 9   high_24h                          1 non-null      int64  
 10  low_24h                           1 non-null      int64  
 11  price_change_24h                  1 non-null      float64
 12  price_change

In [581]:
coins_df = coins_df.drop(columns=['image', 'fully_diluted_valuation', 'price_change_24h', 'market_cap_change_24h', 'total_supply', 'roi'])
coins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                1 non-null      object 
 1   symbol                            1 non-null      object 
 2   name                              1 non-null      object 
 3   current_price                     1 non-null      int64  
 4   market_cap                        1 non-null      int64  
 5   market_cap_rank                   1 non-null      int64  
 6   total_volume                      1 non-null      int64  
 7   high_24h                          1 non-null      int64  
 8   low_24h                           1 non-null      int64  
 9   price_change_percentage_24h       1 non-null      float64
 10  market_cap_change_percentage_24h  1 non-null      float64
 11  circulating_supply                1 non-null      float64
 12  max_supply  

In [582]:
coins_df['ath_date'] = pd.to_datetime(coins_df['ath_date'], errors='coerce')
coins_df['atl_date'] = pd.to_datetime(coins_df['atl_date'], errors='coerce')
coins_df['last_updated'] = pd.to_datetime(coins_df['last_updated'], errors='coerce')

decimals_dict = {
    'current_price': 8,
    'high_24h': 8,
    'low_24h': 8,
    'price_change_percentage_24h': 4,
    'market_cap_change_percentage_24h': 4,
    'circulating_supply': 8,
    'max_supply': 8,
    'ath': 8,
    'ath_change_percentage': 4,
    'atl': 8,
    'atl_change_percentage': 4
}

coins_df = coins_df.round(decimals_dict)

coins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype              
---  ------                            --------------  -----              
 0   id                                1 non-null      object             
 1   symbol                            1 non-null      object             
 2   name                              1 non-null      object             
 3   current_price                     1 non-null      int64              
 4   market_cap                        1 non-null      int64              
 5   market_cap_rank                   1 non-null      int64              
 6   total_volume                      1 non-null      int64              
 7   high_24h                          1 non-null      int64              
 8   low_24h                           1 non-null      int64              
 9   price_change_percentage_24h       1 non-null      float64            

# LOAD

In [583]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [584]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")


[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [585]:
sql_table = "market_snapshots"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")
    

[SUCCESS] Table 'market_snapshots' exists in database 'cryptodb'


In [586]:
UPSERT_SQL = f"""
INSERT INTO {sql_table} (
    coin_id, symbol, name, current_price, market_cap, 
    market_cap_rank, total_volume, high_24h, low_24h, 
    price_change_percentage_24h, market_cap_change_percentage_24h, 
    circulating_supply, max_supply, ath, ath_change_percentage, 
    ath_date, atl, atl_change_percentage, atl_date, last_updated
) 
VALUES (
    %s, %s, %s, %s, %s, 
    %s, %s, %s, %s, 
    %s, %s, 
    %s, %s, %s, %s, 
    %s, %s, %s, %s, %s
) AS src
ON DUPLICATE KEY UPDATE
    coin_id = src.coin_id,
    symbol = src.symbol,
    name = src.name,
    current_price = src.current_price,
    market_cap = src.market_cap,
    market_cap_rank = src.market_cap_rank,
    total_volume = src.total_volume,
    high_24h = src.high_24h,
    low_24h = src.low_24h,
    price_change_percentage_24h = src.price_change_percentage_24h,
    market_cap_change_percentage_24h = src.market_cap_change_percentage_24h,
    circulating_supply = src.circulating_supply,
    max_supply = src.max_supply,
    ath = src.ath,
    ath_change_percentage = src.ath_change_percentage,
    ath_date = src.ath_date,
    atl = src.atl,
    atl_change_percentage = src.atl_change_percentage,
    atl_date = src.atl_date,
    last_updated = src.last_updated
"""


In [587]:
coins_list = coins_df.values.tolist()
print(coins_list)

[['bitcoin', 'btc', 'Bitcoin', 117167, 2334064704577, 1, 41488303467, 117888, 116774, -0.0106, 0.1613, 19922890.0, 21000000.0, 124128, -5.6328, Timestamp('2025-08-14 00:37:02.582000+0000', tz='UTC'), 67.81, 172644.3247, Timestamp('2013-07-06 00:00:00+0000', tz='UTC'), Timestamp('2025-09-19 03:04:23.081000+0000', tz='UTC')]]


In [588]:
try:
    db_cur.executemany(UPSERT_SQL, coins_list)
    db_conn.commit()
    print(f"[SUCCESS] Upserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to upsert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Upserted 1 records into table 'market_snapshots'
[INFO] MySQL connection closed
